In [ ]:
"""
Created by Perrine Bauchot.

#Retrieval of barocline and barotropic contributions to the global SSH from an altimetric track by applying a Lancsoz filter.
"""


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set up

In [ ]:
import netCDF4 as nc
import numpy as np
import numpy.ma as npma
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import math
import scipy.interpolate as sci
import cmath
import xarray as xr

### MISSION ###
mission='TOPEX'
pas = 5.77

def low_pass_weights(window, cutoff):
    """
    Calculate weights for a low pass Lancsoz filter
    :param window: longueur de la fenêtre de filtrage
    :param cutoff: fréquence de coupure
    :return: weights for a low-pass filter of Lancsoz

    """
    order = ((window - 1) // 2) +1
    nwts = 2*order +1
    w = np.zeros([nwts])
    n = nwts //2
    w[n] = 2*cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi*k/n)*n/(np.pi*k)
    firstfactor = np.sin(2.* np.pi*cutoff*k)/(np.pi*k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma

    return w[1:-1]

def lancsoz_filter(data, window, cutoff):
    """
    Effectue un filtrage de Lancsoz passe-bas

    :param data: données à filtrer
    :param lat: dimension (temporelle ou spatiale)
    :param window: longueur de la fenêtre de filtrage
    :param cutoff: fréquence de coupure

    :return: données filtrées
    """

    #Calcul des poids pour le filtrage
    weights = low_pass_weights(window,cutoff)
    w = xr.DataArray(weights, dims = ['window'])

    #Données sous forme xarray DataFrame
    df = xr.DataArray(data, dims=['time'])

    #Filtrage
    lowpass = df.rolling(time=len(weights), center = True)
    lowpass = lowpass.construct('window')
    lowpass = lowpass.dot(w, dims='window')
    return(lowpass)

def find_lat(latitude, latMax, latMin):
    """
    Permet de trouver les indices du tableau pour lesquels la latitude est comprise
    entre latMin et latMax

    Input :
        Tableau latitude
    Output :
        int Lat_deb  Lat_fin
    """
    Lat_fin = 0
    Lat_deb = 0
    i = 0
    if float(latitude[0]) < float(latitude[1]):  # Si on va dans l'ordre croissant
        while float(latitude[i]) < float(latMax):
            i += 1
        Lat_fin = i
    else:  # Si on va dans l'ordre decroissant
        while float(latitude[i]) > float(latMax):
            i += 1
        Lat_deb = i

    i = 0
    if float(latitude[0]) < float(latitude[1]):  # Si on va dans l'ordre croissant
        while float(latitude[i]) < float(latMin):
            i += 1
        Lat_deb = i
    else:  # Si on va dans l'ordre decroissant
        while float(latitude[i]) > float(latMin):
            i += 1
        Lat_fin = i

    return (Lat_deb, Lat_fin)

# Filtrage

In [ ]:
### OBSERVATIONS ###
obs = nc.Dataset('./Residus_t137.mgr.nc')
lat_obs = obs.variables['lat'][:]
lon_obs = obs.variables['lon'][:]
amp_obs = obs.variables['amplitude'][:, 0] * 100
ph_obs = obs.variables['phase_lag'][:, 0]
lat_obs.mask = npma.nomask
lon_obs.mask = npma.nomask
amp_obs.mask = npma.nomask
ph_obs.mask = npma.nomask

n=0
### TRACE ASCENDANTE ou DESCENDANTE ###
if lat_obs[0]<=lat_obs[10]:
    print('Trace ascendante')
else:
    print('Trace descendante')
    amp_obs = list(amp_obs)
    ph_obs = list(ph_obs)
    lat_obs = list(lat_obs)
    lon_obs = list(lon_obs)
    amp_obs.reverse()
    ph_obs.reverse()
    lat_obs.reverse()
    lon_obs.reverse()

### CALCUL PARTIE REELLE ET IMAGINAIRE ###
re = []
im = []
for k in range(len(amp_obs)):
    r = amp_obs[k] * np.cos(-ph_obs[k] * np.pi / 180)
    i = amp_obs[k] * np.sin(-ph_obs[k] * np.pi / 180)
    re.append(r)
    im.append(i)

### REMPLISSAGE DES TROUS CAUSES PAR LA PRESENCE D ILES/CONTINENTS ###
elev_r_comble = []
elev_i_comble = []
lat_comble = []
lon_comble = []
Nb_Pts_comble = 0
holes = []
nbpts = len(re)
R = 6371  # Earth radius km

for j in np.arange(nbpts - 1):

    if j == 0:
        elev_r_comble.append(re[j])
        elev_i_comble.append(im[j])
        lat_comble.append(lat_obs[j])
        lon_comble.append(lon_obs[j])

    # Distance computation between two consecutives points
    dist = np.abs(2 * R * np.arcsin(np.sqrt(
        pow(np.sin((lat_obs[j] * np.pi / 180 - lat_obs[j + 1] * np.pi / 180) / 2), 2) + np.cos(lat_obs[j + 1] * np.pi / 180) * np.cos(
            lat_obs[j] * np.pi / 180) * pow(np.sin((lon_obs[j] * np.pi / 180 - lon_obs[j + 1] * np.pi / 180) / 2), 2))))

    # Distance < pas: on récupère les données de la trace
    if abs(dist - pas) < 1.:
        Nb_Pts_comble = Nb_Pts_comble + 1
        elev_r_comble.append(re[j + 1])
        elev_i_comble.append(im[j + 1])
        lat_comble.append(lat_obs[j + 1])
        lon_comble.append(lon_obs[j + 1])

    # Distance > pas: on interpole linéairement pour combler le trou
    else:
        # Create new points in latitude
        print('####### Create new points')

        Nb = math.floor(dist / pas)
        dlat = (lat_obs[j + 1] - lat_obs[j]) / Nb

        print('Missing point for this hole : ', Nb - 1)

        lat_new = []
        lon_new = []
        elev_r_new = []
        elev_i_new = []

        for k in np.arange(Nb - 1):

            Nb_Pts_comble = Nb_Pts_comble + 1
            lat_new.append(lat_obs[j] + dlat * (k + 1))
            elev_r_new.append(0.)
            elev_i_new.append(0.)
            lon_new.append(0.)

        # Interpolation on the new points

        X = [lat_obs[j], lat_obs[j + 1]]

        Y_lon = [lon_obs[j], lon_obs[j + 1]]
        f_lon = sci.interp1d(X, Y_lon)
        lon_new = f_lon(lat_new)

        Y_elev_r = [re[j], re[j + 1]]
        f_elev_r = sci.interp1d(X, Y_elev_r)
        elev_r_new = f_elev_r(lat_new)

        Y_elev_i = [im[j], im[j + 1]]
        f_elev_i = sci.interp1d(X, Y_elev_i)
        elev_i_new = f_elev_i(lat_new)

        # Add fill data
        print('###### Fill holes')
        for l in range(len(lat_new)):
            elev_r_comble.append(elev_r_new[l])
            elev_i_comble.append(elev_i_new[l])
            lat_comble.append(lat_new[l])
            lon_comble.append(lon_new[l])
            holes.append(lat_new[l])

        # Add last point
        elev_r_comble.append(re[j + 1])
        elev_i_comble.append(im[j + 1])
        lat_comble.append(lat_obs[j + 1])
        lon_comble.append(lon_obs[j + 1])

### FILTRAGE LANCSOZ ###
fs = 1 / pas
nyq = 0.5 * fs
low = 1 / 500.
low = low / nyq

#Recherche des trous consécutifs pour séparer le filtrage des traces trouées par le continent
idx_holes = [np.where(lat_comble == h)[0][0] for h in holes]
"""
n_list=[]
N = 0
for i in range(len(idx_holes)):
    if int(idx_holes[i]-1) == idx_holes[i-1]:
        N+=1
    else:
        n_list.append(N)
        N=0
n_list.append(N)
print(f + ': ' + str(max(n_list)))
if max(n_list) >= 125:
    window=1000
else:
    window=2000
"""
window = 1400


#Application du filtre sur les parties réelles et imaginaires du signal
signal_re_l = lancsoz_filter(elev_r_comble, window, low)
signal_im_l = lancsoz_filter(elev_i_comble, window, low)

# Convert filled holes to NaN --------
signal_re_l[idx_holes] = np.nan
signal_im_l[idx_holes] = np.nan

#Séparation marée barocline/barotrope
lancsoz_re=[]
lancsoz_im=[]
for k in range(len(signal_re_l)):
    lancsoz_re.append(elev_r_comble[k] - float(signal_re_l[k]))
    lancsoz_im.append(elev_i_comble[k] - float(signal_im_l[k]))

### RECALCUL DE L'AMPLITUDE ET DE LA PHASE ###
amp_l=[]
amp_b=[]
ph_l = []
phase_baro = []
for k in range(len(signal_re_l)):
    amp_l.append(np.sqrt(lancsoz_re[k] ** 2 + lancsoz_im[k] ** 2))
    amp_b.append(np.sqrt(signal_re_l[k]**2 + signal_im_l[k]**2))
    cplx_l = lancsoz_re[k] - 1j*lancsoz_im[k]
    cplx_b = signal_re_l[k] - 1j*signal_im_l[k]
    ph_l.append(-np.angle(cplx_l, deg=True))
    phase_baro.append(-np.angle(cplx_b,deg=True))

### SAUVEGARDE DU FICHIER ###
size = len(lat_comble)
output = nc.Dataset('./Residus_t137_filtre.nc', 'w')
output.createDimension('records', size=size)

lat = output.createVariable('lat', 'f', ('records',))
lat.units = 'degrees_north'
lat.standard_name = 'latitude'
lat.axis = 'Y'
lat[:] = lat_comble

lon = output.createVariable('lon', 'f', ('records',))
lon.units = 'degrees_east'
lon.standard_name = 'longitude'
lon.axis = 'X'
lon[:] = lon_comble

amplitudes_h = output.createVariable('amplitude_h', 'f', ('records',))
amplitudes_h.units = 'cm'
amplitudes_h.standard_name = 'amplitude M2 barocline'
amplitudes_h[:] = amp_l[:]

amp_h_reelle = output.createVariable('amplitude_h_reelle', 'f', ('records',))
amp_h_reelle.units = 'cm'
amp_h_reelle.standard_name = 'amplitude M2 barocline reelle'
amp_h_reelle[:] = lancsoz_re

amp_h_im = output.createVariable('amplitude_h_imaginaire', 'f', ('records',))
amp_h_reelle.units = 'cm'
amp_h_reelle.standard_name = 'amplitude M2 barocline imaginaire'
amp_h_reelle[:] = lancsoz_im

amplitudes_b = output.createVariable('amplitude_b', 'f', ('records',))
amplitudes_b.units = 'cm'
amplitudes_b.standard_name = 'amplitude M2 barotrope'
amplitudes_b[:] = amp_b[:]

amp_b_reelle = output.createVariable('amplitude_b_reelle', 'f', ('records',))
amp_b_reelle.units = 'cm'
amp_b_reelle.standard_name = 'amplitude M2 barotrope reelle'
amp_b_reelle[:] = signal_re_l[:]

amp_b_im = output.createVariable('amplitude_b_imaginaire', 'f', ('records',))
amp_b_reelle.units = 'cm'
amp_b_reelle.standard_name = 'amplitude M2 barotrope imaginaire'
amp_b_reelle[:] = signal_im_l[:]

phases = output.createVariable('phase_h', 'f', ('records',))
phases.units = 'degrees'
phases.standard_name = 'phase M2 de la marée barocline'
phases[:] = ph_l

phases_b = output.createVariable('phase_b', 'f', ('records',))
phases_b.units = 'degrees'
phases_b.standard_name = 'phase M2 de la marée barotrope'
phases_b[:] = phase_baro

output.close()

n+=1


Trace descendante
####### Create new points
Missing point for this hole :  1
###### Fill holes
####### Create new points
Missing point for this hole :  4
###### Fill holes
####### Create new points
Missing point for this hole :  1
###### Fill holes
####### Create new points
Missing point for this hole :  3
###### Fill holes
####### Create new points
Missing point for this hole :  397
###### Fill holes
